In [ ]:
print("hello")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from datasets import load_dataset
dataset_name = "idopinto/rlinv_train_sft_test"
dataset = load_dataset(dataset_name, split="train")
print(f"Loaded dataset {dataset_name}...")
print(dataset)

In [ ]:
print(dataset[0])

In [ ]:
from transformers import AutoTokenizer
model_name = "openai/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Loaded tokenizer {tokenizer}...")

In [ ]:
from transformers import AutoModelForCausalLM, Mxfp4Config
quantization_config = Mxfp4Config(dequantize=False)
print(f"Loaded quantization config {quantization_config}...")

In [ ]:
model_kwargs = dict(
    attn_implementation="eager",
    # dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)
model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
print(f"Loaded model {model}...")

In [ ]:
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

model_name = "openai/gpt-oss-20b"
trained_model_name = "gpt-oss-20b-rlinv-sft"
dataset_name = "idopinto/rlinv_train_sft_test"
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir=trained_model_name,
    report_to="wandb",
    push_to_hub=True,
)